In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import openpyxl
import datetime
from scipy.stats import lognorm
import re
import string
from bs4 import BeautifulSoup
import requests
import unicodedata # for removing accented characters

import pdfplumber


In [255]:
os.getcwd()
#os.chdir("/Users/veesheen/Desktop")

'/Users/veesheenyuen/code/veeyuen/SAA'

# Extract results from NSG

In [271]:
if 'Disqualified' in fragment:
    fragment.insert(7, ' ')
    
else:
    fragment.insert(len(fragment), ' ')

In [573]:
import re

pattern='[A-Z]{2,5}'

for i in range(len(fragment)):

    if re.search(pattern, fragment[i]):
        print(i, re.search(pattern, fragment[i]))
        index=i  # set index to second value if there are two uppercase values encountered
        print(index)



1 <re.Match object; span=(0, 4), match='CHIJ'>
1
6 <re.Match object; span=(0, 3), match='SNG'>
6


In [384]:
# Regex pattern testing

#pattern='([A-Z][a-z]+\s){2,4}'

#pattern='\d{1,3}{(?<=\s)[A-Z]{3,5}'

#pattern='[A-Z]{2,4}'

pattern='\d\s\[A-Z]{3,}(?=Q|q|[0-9]{2,3})'


#string='8 CHIJ Sec (Toa Payoh) HIJ 2 7 00:54.31'
#string='999 Chestnut Drive Sec CDS 2 3 Withdrawal'
#string='7 CHIJ Katong Convent KC 5 7 00:54.10'
#string = '1 XIN BOWEN 587 HCI Finals 6 40.15 9'


#matches=re.finditer(pattern, string)

matches=re.findall(pattern, string)

print(matches)

#print(type(matches))

#for match in matches:
#    print(match)



[]


In [126]:
# Test splicing and extracting name

#s = "4 PHOEBE ANG KE WEI Q 800 CGS h4 2 01:05.62"
#s="2 Sarah Chong Xin RI 1 12 4.78 7"
#s='7 CHIJ Katong Convent KC 5 7 00:54.10'
#s='4 WANG HANCHEN 584 HCI Finals 1 36.79 5'
#s='1 XIN BOWEN 587 HCI Finals 6 40.15 9'
#s='3 SINGAPORE SPORTS SCHOOL Q 598 SSP h1 3 04:32.84'
s='16 TEMASEK SECONDARY SCHOOL 873 TMS h4 3 05:08.64'

lpos = re.search('\s', s)
#rpos = re.search('\b[A-Z]{2-4}\b\d', s)

#rpos = re.search('\sQ\s', s)
#rpos = re.search('\s[A-Z]{2,4}?', s)
#rpos = re.search('\s[0-9]{1,3}?|\sQ\s[0-9]{1,3}?|\s[A-Z]{2,4}\s\d?', s)
rpos = re.search('\d\s.{5,40}?\s(Q|q|\d)', s)



split_pos_end=rpos.end()-1
split_pos_start=lpos.start()+1



l = s[:lpos.start()]
r = s[rpos.start():]

#name=s[lpos.start():rpos.start()]

name=s[split_pos_start:split_pos_end]

print(name)

TEMASEK SECONDARY SCHOOL 


In [87]:
# test regex for event name extraction

s='Event: S13-09 Discus - B DIVISION BOYS Finals'
event_pattern=':\s.{5,20}\s\-'


re.search(event_pattern, s)


<re.Match object; span=(5, 22), match=': S13-09 Discus -'>

In [290]:
# Test pdfplumber

os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/sectrack_audit_2010_1-1')
#os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/')



import pdfplumber

file="sectrack_audit_2010_211-211.pdf"
#file="Session 13_13-end.pdf"
#file="Session 13_1-4.pdf"
#file="sectrack_audit_2010_28-28.pdf"
#file="Session 13_5-6.pdf"



#table_settings = {
#    "vertical_strategy": "text",
#    "horizontal_strategy": "text",
#    }


with pdfplumber.open(file) as pdf:
    
    page = pdf.pages[0]
    table=page.extract_table()
    text=page.extract_text()


splitted=text.splitlines()

splitted

['Singapore Schools Sports Council 14-04-2010',
 '51st National Inter-School Track and Field Championships 2010 10:02 PM',
 'Audit List Page 211',
 'Event : 4 X 100 Metres Relay (C-Girls) Type : All Finals',
 '14-16',
 'Record : Singapore Sports School SSP 2004 ET 00:50.53s',
 'Pos Competitor Team Ht Ln Result Pts Remark',
 '1 Singapore Sports School SSP 1 4 00:49.67 9 New Rec',
 "2 CHIJ St Nicholas Girls' School SNG 1 5 00:51.41 7",
 "3 Nanyang Girls' High School NYGH 1 6 00:52.25 6",
 "4 Cedar Girls' Sec School CG 1 3 00:52.90 5",
 '5 CHIJ Sec (Toa Payoh) HIJ 1 7 00:53.88 4',
 '6 CHIJ Katong Convent KC 1 2 00:55.58 3',
 '7 North Vista Sec School NV 1 8 00:55.69 2',
 "8 Raffles Girls' School (Sec) RGS 1 1 00:57.27 1",
 'AppName : Flash2000 Master 2010.04.14a RptName : fmRptA4ListAuditSession']

In [291]:
pd.DataFrame(splitted)

,0
0,Singapore Schools Sports Council 14-04-2010
1,51st National Inter-School Track and Field Cha...
2,Audit List Page 211
3,Event : 4 X 100 Metres Relay (C-Girls) Type : ...
4,14-16
5,Record : Singapore Sports School SSP 2004 ET 0...
6,Pos Competitor Team Ht Ln Result Pts Remark
7,1 Singapore Sports School SSP 1 4 00:49.67 9 N...
8,2 CHIJ St Nicholas Girls' School SNG 1 5 00:51...
9,3 Nanyang Girls' High School NYGH 1 6 00:52.25 6


In [104]:
def old_assign(table):

    global index
    
    exclusions=[]
    
    df=pd.DataFrame(table)
    columns=df.iloc[6,0]
    names=columns.split(' ')

    df2 = pd.DataFrame(columns=names)
    
    new_df = df2.iloc[:, :-2]    # remove last 2 extraneous columns
     
    
    for i in range(7,(len(df)-1)):
            
            
        if not re.search('^\d', table[i]): # skip row if disqualified or withdrawn (does not begin with a digit)

            
            continue

            
        s=table[i]
        
        lpos = re.search('\s', s)
        #rpos = re.search('\sQ\s', s)
        #rpos = re.search('\s[A-Z]{2,4}', s)
        
        matches=re.findall('\s[A-Z]{2,4}', s)  # find how many uppercase shortforms for school
        
        if len(matches)==2:
            rpos=re.search(matches[1], s)
        else:
            rpos = re.search('\s[A-Z]{2,4}', s)

            
        # Splice string to extract name of competitor

            
        l = s[:lpos.start()] # left string post splicing
        r = s[rpos.start():] # right string post slicing
         
        name=s[lpos.start():rpos.start()] # extract competitor name
                
        list=r.split(' ')        

        combined=l+ ',' + name + ', '.join(list)

        row=combined.split(",")
                        
        if len(row)!=6:
            
            n=len(row)-6
            row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns                 
                    
        new_df.loc[len(new_df)] = row

    return new_df

In [852]:
assign(splitted)

/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_84233/4187310370.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[len(new_df)] = row
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_84233/4187310370.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[len(new_df)] = row
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_84233/4187310370.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

,Pos,Competitor,Team,Ht,Ln,Result
0,1,Singapore Sports School,SSP,1,4,00:49.67
1,2,CHIJ St Nicholas Girls' School,SNG,1,5,00:51.41
2,3,Nanyang Girls' High School,NYGH,1,6,00:52.25
3,4,Cedar Girls' Sec School,CG,1,3,00:52.90
4,5,CHIJ Sec (Toa Payoh),HIJ,1,7,00:53.88
5,6,CHIJ Katong Convent,KC,1,2,00:55.58
6,7,North Vista Sec School,NV,1,8,00:55.69
7,8,Raffles Girls' School (Sec),RGS,1,1,00:57.27


In [322]:
def new_format_parser(table, master_df):
    
    df=pd.DataFrame(table)
    
    columns=[]
    
    for i in range(row_index+1,(len(df)-1)):
                                
        if 'DNM' in table[i]:
                    
            continue # skip row if DNM, DQ, DNF etc.
                            
        if not re.search('^\d', table[i]): # look for stranded parts of names (does not begin with a digit)
            
            row[1]=row[1]+table[i] # retrieve previous row and add back stranded string to name
                
            new_df.drop(new_df.tail(1).index,inplace=True) # drop last row with incomplete name
                                        
            new_df.loc[len(new_df)] = row  # add back amended row
                    
            continue  # then skip to next row
                              

        string=table[i]
        
        print(string)
        
                                
        lpos = re.search('\s', string)
        rpos = re.search(new_pattern, string)
                                            
            
        # Splice string to extract name of competitor

        split_pos_end=rpos.end()-1      # adjust the splicing position to only capture the name
        split_pos_start=lpos.start()+1

            
        left_string = string[:lpos.start()] # left string post splicing
        right_string = string[split_pos_end:] # right string post slicing
                         
        name=string[split_pos_start:split_pos_end]  # extract competitor name            
                                
        list=right_string.split(' ')        # put everything coming after the name into a list   
                
                
        if len(list)==6:    # check length of list. If less than 7, append blank fields for Pos and W/G 
                    
            list.extend([' ', ' '])
                    
        elif len(list)==5:
                    
            list.extend([' ', ' '])
                                                            
                
        combined=left_string+ ';' + name +'; '+ '; '.join(list)  # use ';' in case names have a ','

        row=combined.split(';')
                                                                
                
        # check if Q or q between tag and name. If not, fill in with empty space.
        
                
        if row[2].strip()=='Q' or row[2] is None:
                    
            pass
                    
        else:
                    
            row.insert(2, ' ')  # insert a blank field if no Q or q
                    
                            
                
                
            new_df.loc[len(new_df)] = row
                    
                    
                    
        return new_df

In [286]:
def extraction(table, master_df):

    global index
        
        
    # Regex patterns to extract competitor names, event name etc.
    
    #new_pattern='\d\s.{5,20}\s\d' # look for text between two numbers or Q/q
    
    #new_pattern='\d\s[A-Z\s\,]{5,30}\s\d' # look for text between two numbers or Q/q
    
    #new_pattern='^[A-Z\s]*$'
    
    new_pattern='\d\s.{5,40}?\s(Q|q|\d)'
    
    old_pattern='\s[A-Z]{2,4}|\s[0-9]{1,3}?|\sQ\s[0-9]{1,3}?' 
    
    event_pattern='\:\s.{5,30}\s(\-|\()'

    school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
    year_pattern='\d\d\d\d'

    
    # Initialize new df and extract event information
    
    df=pd.DataFrame(table)
    
    columns=[]
    
    
    for index, row in df.iterrows(): # find row containing event details and column names
        
        if 'Event' in row[0]:
            
            event=row[0]
            
            print(row[0])
                        
            epos = re.search(event_pattern, event)
                        
            split_end=epos.end()-1      # adjust the splicing position to only capture the event name
            split_start=epos.start()+1

            event=event[split_start:split_end]
                                    
            
            if 'BOYS' in event or 'Boys' in event:
                gender='Male'
                
            if 'GIRLS' in event or 'Girls' in event:
                gender='Female'    
                
            
            if 'Finals' in event:
                round='Finals'
            
            region='Local'  
            
       
            
        if 'Competitor' in row[0]: # extract column names
            
            row_index=index
            columns=df.iloc[row_index,0]
            names=columns.split(' ')
            
    
    
    
    # Check pdf format and begin extraction of results
    
    if 'W/G' not in columns:  # if older pdf format without a 'W/G column'
            
        df2 = pd.DataFrame(columns=names)        
    
        new_df = df2.iloc[:, :-2]    # remove Remarks and N/R columns
         
        for i in range(row_index+1,(len(df)-1)):                    
            
            if not re.search('^\d', table[i]): # skip row if disqualified or withdrawn (does not begin with a digit)
            
                continue
       
            string=table[i]
                                
            lpos = re.search('\s', string)
                #rpos = re.search('\sQ\s', s)
                #rpos = re.search('\s[A-Z]{2,4}', s)
            
            matches=re.findall(school_pattern, string)  # find how many uppercase shortforms for school
                                
            if len(matches)==2:
                
                rpos=re.search(matches[1], string) # choose second match if there are two 
                    
            else:
                    
                rpos = re.search(old_pattern, string)
                
            
        # Splice string to extract name of competitor

            
            l = string[:lpos.start()] # left string post splicing
            r = string[rpos.start():] # right string post slicing
         
            name=string[lpos.start():rpos.start()] # extract competitor name
                                
            list=r.split(' ')   # split the parts after the name    
            
            combined=l+ ',' + name + ', '.join(list)

            row=combined.split(",")
                        
                        
            if ('W/G' not in names) and len(row)!=6:  # if W/G not in list of columns
            
                n=len(row)-6
                row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns  
            
            elif ('W/G' in names) and len(row)>8:
            
                n=len(row)-8
                row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns  
                                        
            new_df.loc[len(new_df)] = row
            
        return new_df
            
    elif 'W/G' in columns: # new format 
            
            df2 = pd.DataFrame(columns=names)        
    
            new_df = df2.iloc[:, :-2]    # remove 'Remarks' and 'NR' columns        

         
            for i in range(row_index+1,(len(df)-1)):
                                
                if 'DNM' in table[i]:
                    
                    continue # skip row if DNM, DQ, DNF etc.
                            
                if not re.search('^\d', table[i]): # look for stranded parts of names (does not begin with a digit)
            
                    row[1]=row[1]+table[i] # retrieve previous row and add back stranded string to name
                
                    new_df.drop(new_df.tail(1).index,inplace=True) # drop last row with incomplete name
                                        
                    new_df.loc[len(new_df)] = row  # add back amended row
                    
                    continue  # then skip to next row
                              

                string=table[i]
                                
                lpos = re.search('\s', string)
                rpos = re.search(new_pattern, string)
                                            
            
        # Splice string to extract name of competitor

                split_pos_end=rpos.end()-1      # adjust the splicing position to only capture the name
                split_pos_start=lpos.start()+1

            
                left_string = string[:lpos.start()] # left string post splicing
                right_string = string[split_pos_end:] # right string post slicing
                         
                name=string[split_pos_start:split_pos_end]  # extract competitor name            
                                
                list=right_string.split(' ')        # put everything coming after the name into a list   
                
                
                if len(list)==6:    # check length of list. If less than 7, append blank fields for Pos and W/G 
                    
                    list.extend([' ', ' '])
                    
                elif len(list)==5:
                    
                    list.extend([' ', ' '])
                                                            
                
                combined=left_string+ ';' + name +'; '+ '; '.join(list)  # use ';' in case names have a ','

                row=combined.split(';')
                                                                
                
        # check if Q or q between tag and name. If not, fill in with empty space.
        
                
                if row[2].strip()=='Q' or row[2] is None:
                    
                    pass
                    
                else:
                    
                    row.insert(2, ' ')  # insert a blank field if no Q or q
                    
                            
                
                
                new_df.loc[len(new_df)] = row
                    
                    
                    
            return new_df

In [288]:
master_df=None

master_df=extraction(splitted, master_df)

Event: S13-03 4 X 400m Relay - B DIVISION GIRLS Heats


/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_89141/430372517.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[len(new_df)] = row
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_89141/430372517.py:203: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.loc[len(new_df)] = row
/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_89141/430372517.py:148: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a

In [289]:
master_df

,POS,Competitor,Q,Tag,Team,Heat,Ln,Result,Pts,W/G
0,1,SINGAPORE SPORTS SCHOOL,Q,720,SSP,h3,3,04:23.41,,
1,2,NANYANG GIRLS' HIGH SCHOOL,Q,1022,NYGH,h3,6,04:24.02,,
2,3,CRESCENT GIRLS' SCHOOL,Q,791,CGS,h3,2,04:24.09,,
3,4,CEDAR GIRLS' SECONDARY SCHOOL,Q,871,CG,h2,4,04:25.16,,
4,5,CHIJ ST. NICHOLAS GIRLS' SCHOOL (SECONDARY),Q,656,SNG,h2,5,04:30.99,,
5,6,NATIONAL JUNIOR COLLEGE,Q,810,NJC,h2,8,04:32.69,,
6,7,DUNMAN HIGH SCHOOL,Q,930,DHS,h2,2,04:37.82,,
7,8,CHIJ SECONDARY (TOA PAYOH),Q,747,HIJ,h3,5,04:39.52,,
8,9,RAFFLES GIRLS' SCHOOL (SECONDARY),Q,845,RGS,h2,1,04:40.60,,
9,10,AHMAD IBRAHIM SECONDARY SCHOOL,,648,AIS,h1,6,04:43.48,,


In [341]:
# Extract list of primary and secondary schools
# Using BS4

URL ='https://en.wikipedia.org/wiki/List_of_secondary_schools_in_Singapore'
page=requests.get(URL)

soup=BeautifulSoup(page.content, 'html.parser')


In [342]:
# Creating list with all tables
tables = soup.find_all('table')

#  Looking for the table with the classes 'wikitable' and 'sortable'
table = soup.find('table', class_='wikitable')

In [343]:
for a in soup.find_all('table'):
    print(a.text)




Name
Type
SchoolCode
Area[4]
Notes / Affiliations (Admissions / Academic Programmes)
Website


Admiralty Secondary School
Government
3072
Woodlands

[1]


Ahmad Ibrahim Secondary School
Government
3201
Yishun

[2]


Anderson Secondary School
GovernmentAutonomous
3001
Ang Mo Kio

[3]


Anglican High School
Government-aidedAutonomousSAP

7101
Bedok

[4]


Anglo-Chinese School (Barker Road)
Government-aided
7032
Newton
Affiliated to:
Methodist Girls' School (Secondary)

[5]


Anglo-Chinese School (Independent)
IndependentIP
IP:9161Express:7001

Queenstown

Offers the International Baccalaureate certificate
Affiliated to:
Methodist Girls' School (Secondary) (IP)

[6]


Ang Mo Kio Secondary School
Government
3026
Ang Mo Kio

[7]


Assumption English School
Government-aided
7002
Bukit Panjang

Affiliated to:
Catholic Junior College (Non-IP)[5]

[8]


Bartley Secondary School
Government
3002
Toa Payoh

[9]


Beatty Secondary School
Government
3003
Toa Payoh

[10]


Bedok Green Secondary Sch

In [364]:
# Extract list of schools using Pandas read_html

sschool_html = pd.read_html("List of secondary schools in Singapore - Wikipedia.html")
pschool_html = pd.read_html("List of primary schools in Singapore - Wikipedia.html")

In [357]:
sschool_html[0]['Name']

0              Admiralty Secondary School
1          Ahmad Ibrahim Secondary School
2               Anderson Secondary School
3                    Anglican High School
4      Anglo-Chinese School (Barker Road)
                      ...                
140                Yuhua Secondary School
141          Yusof Ishak Secondary School
142               Yuying Secondary School
143             Zhenghua Secondary School
144             Zhonghua Secondary School
Name: Name, Length: 145, dtype: object

In [376]:
sec_schools=sschool_html[0]['Name'].tolist()

In [377]:
sec_schools

['Admiralty Secondary School',
 'Ahmad Ibrahim Secondary School',
 'Anderson Secondary School',
 'Anglican High School',
 'Anglo-Chinese School (Barker Road)',
 'Anglo-Chinese School (Independent)',
 'Ang Mo Kio Secondary School',
 'Assumption English School',
 'Bartley Secondary School',
 'Beatty Secondary School',
 'Bedok Green Secondary School',
 'Bedok South Secondary School',
 'Bedok View Secondary School',
 'Bendemeer Secondary School',
 'Boon Lay Secondary School',
 'Bowen Secondary School',
 'Broadrick Secondary School',
 'Bukit Batok Secondary School',
 'Bukit Merah Secondary School',
 'Bukit Panjang Government High School',
 'Bukit View Secondary School',
 'Catholic High School',
 'Canberra Secondary School',
 "Cedar Girls' Secondary School",
 'Changkat Changi Secondary School',
 'CHIJ Katong Convent (Secondary)',
 'CHIJ Secondary (Toa Payoh)',
 "CHIJ St. Joseph's Convent",
 "CHIJ St. Nicholas Girls' School",
 "CHIJ St. Theresa's Convent",
 'Chua Chu Kang Secondary School',
 

In [367]:
pschool_html[1]

,Name,Funding,Type,Area[3],Notes,Website,School Code
0,Admiralty Primary School,Government,Mixed,Woodlands,NaN,[1],1744
1,Ahmad Ibrahim Primary School,Government,Mixed,Yishun,NaN,[2],1738
2,Ai Tong School,"Government-aided, SAP",Mixed,Bishan,Affiliated to Singapore Hokkien Huay Kuan[4],[3],5625
3,Alexandra Primary School,Government,Mixed,Bukit Merah,NaN,[4],1266
4,Anchor Green Primary School,Government,Mixed,Sengkang,NaN,[5],1254
...,...,...,...,...,...,...,...
180,Yuhua Primary School,Government,Mixed,Jurong East,NaN,[180],1656
181,Yumin Primary School,Government,Mixed,Tampines,NaN,[181],1219
182,Zhangde Primary School,Government,Mixed,Bukit Merah,NaN,[182],1199
183,Zhenghua Primary School,Government,Mixed,Bukit Panjang,NaN,[183],1240


In [374]:
prim_schools=pschool_html[1]['Name'].tolist()

In [375]:
prim_schools

['Admiralty Primary School',
 'Ahmad Ibrahim Primary School',
 'Ai Tong School',
 'Alexandra Primary School',
 'Anchor Green Primary School',
 'Anderson Primary School',
 'Anglo-Chinese School (Junior)',
 'Anglo-Chinese School (Primary)',
 'Angsana Primary School',
 'Ang Mo Kio Primary School',
 'Beacon Primary School',
 'Bedok Green Primary School',
 'Bendemeer Primary School',
 'Blangah Rise Primary School',
 'Boon Lay Garden Primary School',
 'Bukit Panjang Primary School',
 'Bukit Timah Primary School',
 'Bukit View Primary School',
 'Canberra Primary School',
 'Canossa Catholic Primary School',
 'Cantonment Primary School',
 'Casuarina Primary School',
 'Catholic High School (Primary)',
 'Cedar Primary School',
 'Changkat Primary School',
 'CHIJ (Katong) Primary',
 'CHIJ (Kellock)',
 'CHIJ Our Lady of Good Counsel',
 'CHIJ Our Lady of the Nativity',
 'CHIJ Our Lady Queen of Peace',
 'CHIJ Primary (Toa Payoh)',
 "CHIJ St. Nicholas Girls' School (Primary Section)",
 'Chongfu School'

In [378]:
all_schools = prim_schools+sec_schools

In [379]:
all_schools

['Admiralty Primary School',
 'Ahmad Ibrahim Primary School',
 'Ai Tong School',
 'Alexandra Primary School',
 'Anchor Green Primary School',
 'Anderson Primary School',
 'Anglo-Chinese School (Junior)',
 'Anglo-Chinese School (Primary)',
 'Angsana Primary School',
 'Ang Mo Kio Primary School',
 'Beacon Primary School',
 'Bedok Green Primary School',
 'Bendemeer Primary School',
 'Blangah Rise Primary School',
 'Boon Lay Garden Primary School',
 'Bukit Panjang Primary School',
 'Bukit Timah Primary School',
 'Bukit View Primary School',
 'Canberra Primary School',
 'Canossa Catholic Primary School',
 'Cantonment Primary School',
 'Casuarina Primary School',
 'Catholic High School (Primary)',
 'Cedar Primary School',
 'Changkat Primary School',
 'CHIJ (Katong) Primary',
 'CHIJ (Kellock)',
 'CHIJ Our Lady of Good Counsel',
 'CHIJ Our Lady of the Nativity',
 'CHIJ Our Lady Queen of Peace',
 'CHIJ Primary (Toa Payoh)',
 "CHIJ St. Nicholas Girls' School (Primary Section)",
 'Chongfu School'

In [242]:
# Test scan of directory files and opening

directory = r"/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6_1-1/"
 
# Iterate over files in directory
for filename in os.scandir(directory):
    # Open file
    with open(os.path.join(directory, filename)) as f:
        print(f"Content of '{filename}'")    
    
    with pdfplumber.open(file) as pdf:
    
        page = pdf.pages[0]
        table=page.extract_table()
        text=page.extract_text()


    splitted=text.splitlines()

    splitted

Content of '<DirEntry 'Session 13_5-6_2-end.pdf'>'
Content of '<DirEntry 'Session 13_5-6_1-1.pdf'>'


In [243]:
splitted

['Audited List',
 'Event: S13-03 4 X 400m Relay - B DIVISION GIRLS Heats',
 'Schools National Record Singapore Sports School SSP (ET)04:01.98s 2011',
 'Championships Record Singapore Sports School SSP (ET)04:01.98s 2011',
 'POS Competitor Q Tag Team Heat Ln Result Pts W/G Remarks NR',
 '1 SINGAPORE SPORTS SCHOOL Q 720 SSP h3 3 04:23.41',
 "2 NANYANG GIRLS' HIGH Q 1022 NYGH h3 6 04:24.02",
 'SCHOOL',
 "3 CRESCENT GIRLS' SCHOOL Q 791 CGS h3 2 04:24.09",
 "4 CEDAR GIRLS' SECONDARY Q 871 CG h2 4 04:25.16",
 'SCHOOL',
 "5 CHIJ ST. NICHOLAS GIRLS' Q 656 SNG h2 5 04:30.99",
 'SCHOOL (SECONDARY)',
 '6 NATIONAL JUNIOR COLLEGE Q 810 NJC h2 8 04:32.69',
 '7 DUNMAN HIGH SCHOOL Q 930 DHS h2 2 04:37.82',
 '8 CHIJ SECONDARY (TOA PAYOH) Q 747 HIJ h3 5 04:39.52',
 "9 RAFFLES GIRLS' SCHOOL Q 845 RGS h2 1 04:40.60",
 '(SECONDARY)',
 '10 AHMAD IBRAHIM SECONDARY 648 AIS h1 6 04:43.48',
 'SCHOOL',
 '11 PAYA LEBAR METHODIST 953 PLMGS h3 1 04:48.28',
 "GIRLS' SCHOOL (SECONDARY)",
 '12 NORTH VISTA SECONDARY 70

In [245]:
def sort_directory(directory):
    items = os.listdir(directory)
    sorted_items = sorted(items)
    return sorted_items

In [246]:
sort_directory(directory)

['Session 13_5-6_1-1.pdf', 'Session 13_5-6_2-end.pdf']

In [325]:
# Test scan of directory files and opening


os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6_1-1/')

directory = r"/Users/veesheenyuen/Desktop/DataScience/SAA/NSG/Session 13_1-4/Session 13_5-6_1-1/"
 
# Iterate over files in directory

sorted_items=sort_directory(directory)

master_df=pd.DataFrame()

for file in sorted_items:
    
    print(file)

    with pdfplumber.open(file) as pdf:
    
        page = pdf.pages[0]
        table=page.extract_table()
        text=page.extract_text()


    splitted=text.splitlines()
    
    master_df=extraction(splitted, master_df)
    print(master_df)

Session 13_5-6_1-1.pdf


NameError: name 'row_index' is not defined

In [275]:
splitted

['23 BUKIT VIEW SECONDARY 999 BTV h3 4 05:51.77', 'SCHOOL', '/']

In [296]:
master_df

In [323]:
def extraction(table, master_df):

    global index
        
        
    # Regex patterns to extract competitor names, event name etc.
    
    #new_pattern='\d\s.{5,20}\s\d' # look for text between two numbers or Q/q
    
    #new_pattern='\d\s[A-Z\s\,]{5,30}\s\d' # look for text between two numbers or Q/q
    
    #new_pattern='^[A-Z\s]*$'
    
    new_pattern='\d\s.{5,40}?\s(Q|q|\d)'
    
    old_pattern='\s[A-Z]{2,4}|\s[0-9]{1,3}?|\sQ\s[0-9]{1,3}?' 
    
    event_pattern='\:\s.{5,30}\s(\-|\()'

    school_pattern='\s[A-Z]{2,4}'  # find how many uppercase shortforms for school
    
    year_pattern='\d\d\d\d'

    
    # Initialize new df and extract event information
    
    df=pd.DataFrame(table)
    
    columns=[]
    
    
    for index, row in df.iterrows(): # find row containing event details and column names
        
        
        if 'Event' in row[0]:
            
            event=row[0]
            
            print(row[0])
                        
            epos = re.search(event_pattern, event)
                        
            split_end=epos.end()-1      # adjust the splicing position to only capture the event name
            split_start=epos.start()+1

            event=event[split_start:split_end]
                                    
            
            if 'BOYS' in event or 'Boys' in event:
                gender='Male'
                
            if 'GIRLS' in event or 'Girls' in event:
                gender='Female'    
                
            
            if 'Finals' in event:
                round='Finals'
            
            region='Local'  
            
            
       
            
        if 'Competitor' in row[0]: # extract column names
            
            row_index=index
            columns=df.iloc[row_index,0]
            names=columns.split(' ')
            
            
        if 'Competitor' not in row[0] and 'Event' not in row[0]:
                        
            new_df = master_df.iloc[:0]  # initiate empty df and copy column names
            
            new_format_parser(df, master_df)
            
        print(event, gender, round)
            
            
    
    # Check pdf format and begin extraction of results
    
    if 'W/G' not in columns:  # if older pdf format without a 'W/G column'
            
        df2 = pd.DataFrame(columns=names)        
    
        new_df = df2.iloc[:, :-2]    # remove Remarks and N/R columns
         
        for i in range(row_index+1,(len(df)-1)):                    
            
            if not re.search('^\d', table[i]): # skip row if disqualified or withdrawn (does not begin with a digit)
            
                continue
       
            string=table[i]
                                
            lpos = re.search('\s', string)
                #rpos = re.search('\sQ\s', s)
                #rpos = re.search('\s[A-Z]{2,4}', s)
            
            matches=re.findall(school_pattern, string)  # find how many uppercase shortforms for school
                                
            if len(matches)==2:
                
                rpos=re.search(matches[1], string) # choose second match if there are two 
                    
            else:
                    
                rpos = re.search(old_pattern, string)
                
            
        # Splice string to extract name of competitor

            
            l = string[:lpos.start()] # left string post splicing
            r = string[rpos.start():] # right string post slicing
         
            name=string[lpos.start():rpos.start()] # extract competitor name
                                
            list=r.split(' ')   # split the parts after the name    
            
            combined=l+ ',' + name + ', '.join(list)

            row=combined.split(",")
                        
                        
            if ('W/G' not in names) and len(row)!=6:  # if W/G not in list of columns
            
                n=len(row)-6
                row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns  
            
            elif ('W/G' in names) and len(row)>8:
            
                n=len(row)-8
                row = row[: len(row) - n]  # delete last n elements from list to makt it 6 columns  
                                        
            new_df.loc[len(new_df)] = row
            
        return new_df
            
    elif 'W/G' in columns: # new format 
                    
            new_format_parser(df, master_df)
            
            return master_df